### Change Log
* Version 1: Copied from original notebook
* Version 2: train_nn created here; added holdout set
* Version 3: Moved the creation of holdout set to training notebook
* Version 5: Cleaned for only test processing
* Version 6: Added min max scaler for NN dataset
* Version 7:
    - Prepare TabNet data inside this notebook
    - Design synthetic test set for debugging submission scoring error
* Version 8: Stock clustering (categorical column)
* Version 10: NN label encoding for embedding
* Version 12: Time clustering (categorical column)
* Version 13: Time clustering - only log transform positive columns
* Version 14: Added velocity features
* Version 15: Added Time Series AutoEncoding features
* Version 18: Error handling re kaggle staff reminder (ERROR / unused)
* Version 20: Removed >0.99 correlated features (ERROR / unused)
* Version 23: Error handling re kaggle staff reminder
* Version 24: Removed >0.99 correlated features
   

### To do list:
* Add skew and kurt and add to stock level aggregation
* Add velocity
* Wavelength and Amplitudes
* Time clustering?
* TS clustering by AE
* Prepare TabNet data inside this notebook
* Time features - only log-transform non-negative columns

In [1]:
TRAIN = True
TEST_MODE = 'test'
FE_PATH = '/kaggle/input/volatility-fe-output-version-15'
N_STOCK = 112
SEED = 1111

if TEST_MODE=='test':
    data_path = 'optiver-realized-volatility-prediction'
elif TEST_MODE=='syn':
    data_path = 'optiver-realized-volatility-prediction-synthetic'

In [2]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np # linear algebra
import glob
import os
import gc
import datetime
import pickle

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from sklearn.preprocessing import MinMaxScaler,StandardScaler,LabelEncoder
from sklearn.preprocessing import QuantileTransformer
from sklearn.metrics import r2_score

import matplotlib.pyplot as plt 
import seaborn as sns
import numpy.matlib
pd.set_option('display.max_columns', None)

In [3]:
# df = pd.DataFrame({'a':[1,2,3,4,5], 'b':['a','a','b','b','c']})
# df.groupby('b')['a'].agg(lambda s:s.max()-s.min())

In [4]:
# fill inf and nan of  a dataframe
def fill_inf_nan(df):
    df = df.replace([np.inf, -np.inf], np.nan) # replace Inf with NaN
    df = df.fillna(df.mean()) # replace NaN with mean
    df = df.fillna(0) # if there are still na, fill with zero
    return df

# function to load a book/trade train/test single stock file
def load_single_stock(stock_id, train_test, book_trade):
    path = f'/kaggle/input/{data_path}/{book_trade}_{train_test}.parquet/stock_id={str(stock_id)}'
    filename = os.path.join(path, os.listdir(path)[0])
    return pd.read_parquet(filename)

# Function to calculate first WAP
def calc_wap1(df):
    return (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
def calc_wap2(df):
    return (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
def calc_wap3(df):
    return (df['bid_price1'] * df['bid_size1'] + df['ask_price1'] * df['ask_size1']) / (df['bid_size1'] + df['ask_size1'])
def calc_wap4(df):
    return (df['bid_price2'] * df['bid_size2'] + df['ask_price2'] * df['ask_size2']) / (df['bid_size2'] + df['ask_size2'])

# Function to calculate the log of the return
def log_return(series):
    return np.log(series).diff()

# Calculate the realized volatility
def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

# Function to count unique elements of a series
def count_unique(series):
    return len(np.unique(series))

# function to return range of a series
def min_max_range(s):
    return s.max() - s.min()

# Function to read our base train and test set
def read_train_test():
    train = pd.read_csv(f'../input/{data_path}/train.csv')
    test = pd.read_csv(f'../input/{data_path}/test.csv')
    # Create a key to merge with book and trade data
    train['row_id'] = train['stock_id'].astype(str) + '-' + train['time_id'].astype(str)
    test['row_id'] = test['stock_id'].astype(str) + '-' + test['time_id'].astype(str)
    print(f'Our training set has {train.shape[0]} rows')
    return train, test

# Function to preprocess book data (for each stock id)
def book_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    # Calculate Wap
    df['wap1'] = calc_wap1(df)
    df['wap2'] = calc_wap2(df)
    df['wap3'] = calc_wap3(df)
    df['wap4'] = calc_wap4(df)
    # Calculate log returns
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['log_return3'] = df.groupby(['time_id'])['wap3'].apply(log_return)
    df['log_return4'] = df.groupby(['time_id'])['wap4'].apply(log_return)
    # Calculate wap balance
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    # Calculate spread
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['price_spread2'] = (df['ask_price2'] - df['bid_price2']) / ((df['ask_price2'] + df['bid_price2']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df["bid_ask_spread"] = abs(df['bid_spread'] - df['ask_spread'])
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    # Moving Avereage features
    df['log_return1_sma'] = df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=50).mean())
    df['log_return1_sms'] = df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=50).std())
    df['log_return1_sma_diff'] = df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=20).mean()) - df.groupby('time_id')['log_return1'].apply(lambda s: s.rolling(window=50).mean())
    df['log_return1_sms_diff'] = df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=20).std()) - df.groupby(['time_id'])['log_return1'].apply(lambda s: s.rolling(window=50).std())

    # Dict for aggregations; These are the features that only exist in full seconds range (000-599)
    create_feature_dict = {
        'wap1': [np.sum, np.std],
        'wap2': [np.sum, np.std],
        'wap3': [np.sum, np.std],
        'wap4': [np.sum, np.std],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'wap_balance': [np.sum, np.max],
        'price_spread':[np.sum, np.max],
        'price_spread2':[np.sum, np.max],
        'bid_spread':[np.sum, np.max],
        'ask_spread':[np.sum, np.max],
        'total_volume':[np.sum, np.max],
        'volume_imbalance':[np.sum, np.max],
        "bid_ask_spread":[np.sum,  np.max],
        'log_return1_sma':['last', min_max_range],
        'log_return1_sms':['last', min_max_range],
        'log_return1_sma_diff':['last', min_max_range],
        'log_return1_sms_diff':['last', min_max_range],
    }
    # These are the features that will be replicated for last 500, 400, 300, 200, 100 seconds
    create_feature_dict_time = {
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return3': [realized_volatility],
        'log_return4': [realized_volatility],
        'total_volume': ['sum','max'],
        'volume_imbalance': ['sum','max']
    }
    
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time, seconds_in_bucket = 100, add_suffix = True)

    # Merge all
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    # Drop unnecesary time_ids
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id__100'], axis = 1, inplace = True)
    
    # Create row_id so we can merge
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to preprocess trade data (for each stock id)
def trade_preprocessor(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['amount']=df['price']*df['size']
    # Dict for aggregations
    create_feature_dict = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum, np.max, np.min],
        'order_count':[np.sum,np.max],
        'amount':[np.sum,np.max,np.min],
    }
    # These are the features to be replicated for 500, 400,...100 seconds filter
    create_feature_dict_time = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.sum],
    }
    # Function to get group stats for different windows (seconds in bucket)
    def get_stats_window(fe_dict,seconds_in_bucket, add_suffix = False):
        # Group by the window
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(fe_dict).reset_index()
        # Rename columns joining suffix
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        # Add a suffix to differentiate windows
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    # Get the stats for different windows
    df_feature = get_stats_window(create_feature_dict,seconds_in_bucket = 0, add_suffix = False)
    df_feature_500 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 500, add_suffix = True)
    df_feature_400 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 400, add_suffix = True)
    df_feature_300 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 300, add_suffix = True)
    df_feature_200 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 200, add_suffix = True)
    df_feature_100 = get_stats_window(create_feature_dict_time,seconds_in_bucket = 100, add_suffix = True)
    
    # Create feautres for Tendency and Energy (?); Total 11 features applied to only full seconds (000-599)
    def tendency(price, vol):    
        df_diff = np.diff(price)
        val = (df_diff/price[1:])*100
        power = np.sum(val*vol[1:])
        return(power)
    lis = []
    for n_time_id in df['time_id'].unique():
        df_id = df[df['time_id'] == n_time_id]        
        tendencyV = tendency(df_id['price'].values, df_id['size'].values)      
        f_max = np.sum(df_id['price'].values > np.mean(df_id['price'].values))
        f_min = np.sum(df_id['price'].values < np.mean(df_id['price'].values))
        df_max =  np.sum(np.diff(df_id['price'].values) > 0)
        df_min =  np.sum(np.diff(df_id['price'].values) < 0)
        # new
        abs_diff = np.median(np.abs( df_id['price'].values - np.mean(df_id['price'].values)))        
        energy = np.mean(df_id['price'].values**2)
        iqr_p = np.percentile(df_id['price'].values,75) - np.percentile(df_id['price'].values,25)
        # vol vars
        abs_diff_v = np.median(np.abs( df_id['size'].values - np.mean(df_id['size'].values)))        
        energy_v = np.sum(df_id['size'].values**2)
        iqr_p_v = np.percentile(df_id['size'].values,75) - np.percentile(df_id['size'].values,25)
        lis.append({'time_id':n_time_id,'tendency':tendencyV,'f_max':f_max,'f_min':f_min,'df_max':df_max,'df_min':df_min,
                   'abs_diff':abs_diff,'energy':energy,'iqr_p':iqr_p,'abs_diff_v':abs_diff_v,'energy_v':energy_v,'iqr_p_v':iqr_p_v})
    df_lr = pd.DataFrame(lis)
    df_feature = df_feature.merge(df_lr, how = 'left', left_on = 'time_id_', right_on = 'time_id')
    
    # Merge all seconds filter
    df_feature = df_feature.merge(df_feature_500, how = 'left', left_on = 'time_id_', right_on = 'time_id__500')
    df_feature = df_feature.merge(df_feature_400, how = 'left', left_on = 'time_id_', right_on = 'time_id__400')
    df_feature = df_feature.merge(df_feature_300, how = 'left', left_on = 'time_id_', right_on = 'time_id__300')
    df_feature = df_feature.merge(df_feature_200, how = 'left', left_on = 'time_id_', right_on = 'time_id__200')
    df_feature = df_feature.merge(df_feature_100, how = 'left', left_on = 'time_id_', right_on = 'time_id__100')
    df_feature.drop(['time_id__500','time_id__400', 'time_id__300', 'time_id__200','time_id','time_id__100'], axis = 1, inplace = True)
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

# Function to get group stats for the stock_id and time_id
def get_time_stock(df):
    stock_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility','trade_log_return_realized_volatility']

    time_cols = ['log_return1_realized_volatility', 'log_return2_realized_volatility', 'log_return1_realized_volatility_400', 'log_return2_realized_volatility_400', 
                'log_return1_realized_volatility_300', 'log_return2_realized_volatility_300', 'log_return1_realized_volatility_200', 'log_return2_realized_volatility_200', 
                'trade_log_return_realized_volatility', 'trade_log_return_realized_volatility_400', 'trade_log_return_realized_volatility_300', 'trade_log_return_realized_volatility_200',
                'log_return1_sma_last','log_return1_sma_diff_last','log_return1_sms_last','log_return1_sms_diff_last']
    
    # Group by the stock id
    df_stock_id = df.groupby(['stock_id'])[stock_cols].agg(['mean','min','max','std']).reset_index()
    # Rename columns joining suffix
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    # Group by the time id
    df_time_id = df.groupby(['time_id'])[time_cols].agg(['mean','min','max','std']).reset_index()
    # Rename columns joining suffix
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    # Merge with original dataframe
    df = df.merge(df_stock_id, how = 'left', left_on = ['stock_id'], right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = ['time_id'], right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

# Funtion to make preprocessing function in parallel (for each stock id)
def preprocessor(list_stock_ids, mode='train'):
    # Parrallel for loop
    def for_joblib(stock_id):
        print(f'Processing stock {stock_id}')
        # Train
        file_path_book = f'/kaggle/input/{data_path}/book_{mode}.parquet/stock_id={stock_id}'
        file_path_trade = f'/kaggle/input/{data_path}/trade_{mode}.parquet/stock_id={stock_id}'
        # Preprocess book and trade data and merge them
        df_tmp = pd.merge(book_preprocessor(file_path_book), trade_preprocessor(file_path_trade), on = 'row_id', how = 'left')
        return df_tmp
    # Use parallel api to call paralle for loop
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    # Concatenate all the dataframes that return from Parallel
    df = pd.concat(df, ignore_index = True)
    return df

# Function to calculate the root mean squared percentage error
def rmspe(y_true, y_pred):
    return np.sqrt(np.mean(np.square((y_true - y_pred) / y_true)))

# Function to early stop with root mean squared percentage error
def feval_rmspe(y_pred, lgb_train):
    y_true = lgb_train.get_label()
    return 'RMSPE', rmspe(y_true, y_pred), False

In [5]:
%%time
'''
Create all basic features
'''
# Create basic features for train data
if TRAIN==True:
    train, test = read_train_test()
    train_stock_ids = train['stock_id'].unique()[:N_STOCK]
    train = train.merge(preprocessor(train_stock_ids, mode='train'), on = ['row_id'], how = 'left')
    key_cols = ['stock_id','time_id','row_id','target']
    feat_cols = [col for col in train.columns if col not in key_cols]
    train[feat_cols] = train[feat_cols].fillna(train[feat_cols].mean()).fillna(0)
    train = get_time_stock(train)
else:
    train = pd.read_feather(os.path.join(FE_PATH, 'train.f'))

# Create basic features for test data
_ , test = read_train_test()
test_stock_ids = test['stock_id'].unique()[:N_STOCK]
test = test.merge(preprocessor(test_stock_ids, mode='test'), on=['row_id'], how='left')
key_cols = ['stock_id','time_id','row_id','target']
feat_cols = [col for col in test.columns if col not in key_cols]
test[feat_cols] = test[feat_cols].fillna(test[feat_cols].mean()).fillna(0)
test = get_time_stock(test)

Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 20.6min
[Parallel(n_jobs=-1)]: Done 112 out of 112 | elapsed: 52.6min finished


Our training set has 428932 rows


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


CPU times: user 13.6 s, sys: 3.96 s, total: 17.6 s
Wall time: 52min 49s


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.4s finished


In [6]:
%%time
'''
Adding correlated stocks features
'''
# function to get time series for all stocks for similarity calculation in next step
def gen_knn_stock_data(train_test, n_stock, seconds_sample_interval):
    # define parameters
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/{data_path}/book_{train_test}.parquet')])
    stock_list = stock_list[:n_stock]
    # setup base table as all combinations of time x seconds
    time_id_list = pd.DataFrame({'time_id':sorted(load_single_stock(0, train_test, 'book').time_id.unique().tolist())})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    # loop through stocks to get time series data
    ts = []
    for stock_id in stock_list:
        # load data
        book = load_single_stock(stock_id, train_test, 'book')
        # fill NA
        book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
        book = book.ffill().bfill()  
        # create series
        book['wap1'] = (book['bid_price1'] * book['ask_size1'] + book['ask_price1'] * book['bid_size1']) / ( book['bid_size1'] +  book['ask_size1'])
        book['wap1_lr'] = np.log(book['wap1']).diff()
        book['wap1_sms50'] = book.groupby('time_id')['wap1_lr'].apply(lambda s: s.rolling(window=50).std())
        # filtering
        idx = book.index.tolist()[::seconds_sample_interval]
        book = book.iloc[idx,:]
        book = book[book['wap1_sms50'].isnull()==False].reset_index(drop=True)
        # centering
        book['wap1_sms50'] = book['wap1_sms50'] - book['time_id'].map(book.groupby('time_id')['wap1_sms50'].median())
        # merge with master table
        book[stock_id] = book['wap1_sms50']
        book = book[['time_id','seconds_in_bucket',stock_id]].set_index(['time_id','seconds_in_bucket'])
        ts.append(book)
        if stock_list.index(stock_id)%20==0:
            print(f'[{datetime.datetime.now()}] Added {stock_list.index(stock_id)} stocks time series.')
    ts = pd.concat(ts, axis=1).reset_index().drop('seconds_in_bucket', axis=1)
    print(f'[{datetime.datetime.now()}] Total time series size is {int(ts.memory_usage().sum() / 1024**2)} MB')
    return ts

# function to top N correlated stocks per stock per time_id
def gen_corr_stock_mapping(data, n_top, show_distance):
    n_top = min(n_top, len(data.columns.tolist()[1:])-1)
    time_id_list = data.time_id.unique().tolist() 
    mapping = []
    for time_id in time_id_list:
        corr = data[data.time_id==time_id].iloc[:,1:].corr().reset_index().rename(columns={'index':'corr_stock_id'})
        for stock_id in corr.columns.tolist()[1:]:
            df = pd.DataFrame({'nearest_stocks': corr[corr.corr_stock_id!=stock_id].sort_values(by=stock_id, ascending=False)['corr_stock_id'].tolist()[:n_top]})
            if show_distance==True:
                df['nearest_stocks_corr'] = corr[corr.corr_stock_id!=stock_id].sort_values(by=stock_id, ascending=False)[stock_id].tolist()[:n_top]
            df['stock_id'] = stock_id
            df['time_id'] = time_id
            mapping.append(df)
        if time_id_list.index(time_id)%500==0:
            print(f'[{datetime.datetime.now()}] Completed correlation for {time_id_list.index(time_id)} time_id.')
    mapping = pd.concat(mapping, axis=0)
    mapping = mapping[['stock_id','time_id','nearest_stocks'] + [col for col in mapping.columns if col not in ['stock_id','time_id','nearest_stocks']]]
    return mapping

# generate new features based on correlated stocks
def gen_corr_stock_feats(data, mapping, target_cols):
    new_feats = pd.merge(data.rename(columns={'stock_id':'nearest_stocks'})[['nearest_stocks','time_id'] + target_cols],
                        mapping, how='inner', on=['nearest_stocks','time_id']).\
                        groupby(['stock_id','time_id'])[target_cols].\
                        mean().\
                        reset_index()
    new_feats.columns = ['stock_id','time_id'] + [f'{col}_corr' for col in new_feats.columns if col not in ['stock_id','time_id']]
    data = pd.merge(data, new_feats, how='left', on=['stock_id','time_id'])
    return data

# correlated stock features
target_cols = ['log_return1_realized_volatility','log_return1_realized_volatility_300','log_return1_realized_volatility_100',
               'log_return1_sma_last','log_return1_sma_diff_last','log_return1_sms_last','log_return1_sms_diff_last',
               'total_volume_sum','volume_imbalance_sum','trade_size_sum','price_spread_sum']

if TRAIN==True:
    corr_mapping_train = gen_corr_stock_mapping(gen_knn_stock_data(train_test='train', n_stock=N_STOCK, seconds_sample_interval=10), n_top=5, show_distance=False)
    train = gen_corr_stock_feats(data=train, mapping=corr_mapping_train, target_cols=target_cols)

corr_mapping_test = gen_corr_stock_mapping(gen_knn_stock_data(train_test='test', n_stock=N_STOCK, seconds_sample_interval=10), n_top=5, show_distance=False)
test = gen_corr_stock_feats(data=test, mapping=corr_mapping_test, target_cols=target_cols)

[2021-09-27 04:51:33.136873] Added 0 stocks time series.
[2021-09-27 04:52:44.490979] Added 20 stocks time series.
[2021-09-27 04:53:54.329212] Added 40 stocks time series.
[2021-09-27 04:55:04.826318] Added 60 stocks time series.
[2021-09-27 04:56:15.226218] Added 80 stocks time series.
[2021-09-27 04:57:25.912532] Added 100 stocks time series.
[2021-09-27 04:58:05.818057] Total time series size is 181 MB
[2021-09-27 04:58:06.112345] Completed correlation for 0 time_id.
[2021-09-27 05:00:18.597769] Completed correlation for 500 time_id.
[2021-09-27 05:02:31.494279] Completed correlation for 1000 time_id.
[2021-09-27 05:04:45.353627] Completed correlation for 1500 time_id.
[2021-09-27 05:06:57.383135] Completed correlation for 2000 time_id.
[2021-09-27 05:09:09.653840] Completed correlation for 2500 time_id.
[2021-09-27 05:11:24.569412] Completed correlation for 3000 time_id.
[2021-09-27 05:13:37.538963] Completed correlation for 3500 time_id.
[2021-09-27 05:17:02.239526] Added 0 stock

In [7]:
%%time
'''
Adding features for Time Series shape clustering
'''
!pip install ../input/tslearn052/tslearn-0.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl
from tslearn.clustering import TimeSeriesKMeans, KShape
from tslearn.generators import random_walks
from tslearn.utils import to_time_series_dataset
from tslearn.barycenters import dtw_barycenter_averaging
import math
import random

def preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval, remove_first_n_seconds, delete_unused_cols):
    # load data
    book = load_single_stock(stock_id, train_test, 'book')
    # ffill and bfill
    time_id_list =  pd.DataFrame({'time_id':book.time_id.unique().tolist()})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
    book = book.ffill().bfill()
    # sampling time_id
    if n_time_id_sample >= 0:
        time_id_samples = random.sample(book['time_id'].unique().tolist(), n_time_id_sample)
        book = book[book.time_id.isin(time_id_samples)]
    # create time series
    book['wap1'] = (book['bid_price1'] * book['ask_size1'] + book['ask_price1'] * book['bid_size1']) / ( book['bid_size1'] +  book['ask_size1'])
    book['wap1_sma50'] = book.groupby('time_id')['wap1'].apply(lambda s: s.rolling(window=50).mean())
    book['wap1_lr'] = np.log(book['wap1']).diff()
    book['wap1_sms50'] = book.groupby('time_id')['wap1_lr'].apply(lambda s: s.rolling(window=50).std())
    book['total_volume'] = book['ask_size1'] + book['ask_size2'] + book['bid_size1'] + book['bid_size2']
    book['total_volume'] = (book['total_volume'] - book['total_volume'].mean()) / book['total_volume'].std()
    book['total_volume_sma60'] = book.groupby('time_id')['total_volume'].apply(lambda s: s.rolling(window=60).mean())
    book['volume_imbalance'] = (book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2'])
    book['volume_imbalance'] = (book['volume_imbalance'] - book['volume_imbalance'].mean()) / book['volume_imbalance'].std()
    book['volume_imbalance_sma80'] = book.groupby('time_id')['volume_imbalance'].apply(lambda s: s.rolling(window=80).mean())
    # remove first few entries to avoid NA
    book = book[book.seconds_in_bucket >= remove_first_n_seconds].reset_index(drop=True)
    # seconds interval filtering
    idx = book.index.tolist()[::seconds_sample_interval]
    book = book.iloc[idx,:]
    # select only relevant columns
    book['stock_id'] = stock_id
    if delete_unused_cols==True:
        book = book[['stock_id','time_id','seconds_in_bucket','wap1_sma50','wap1_sms50','total_volume_sma60','volume_imbalance_sma80']]
    return book

def tagging_cluster_all_stocks(n_stock, train_test, n_time_id_sample, seconds_sample_interval, remove_first_n_seconds, delete_unused_cols):
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/{data_path}/book_{train_test}.parquet')])[:n_stock]
    all_stocks_tagging = []
    for stock_id in stock_list:
        ts = preprocess_single_stock(stock_id=stock_id, train_test=train_test, n_time_id_sample=n_time_id_sample, 
                                     seconds_sample_interval=seconds_sample_interval, remove_first_n_seconds=remove_first_n_seconds, 
                                     delete_unused_cols=delete_unused_cols)
        tagging = ts[['stock_id','time_id']].drop_duplicates()
        path = '/kaggle/input/volatility-ts-clustering-models-v1' # define KMeans model path and names
        clust_models = ['clust_wap1_sma50','clust_wap1_sms50','clust_total_volume_sma60','clust_volume_imbalance_sma80']
        for name in clust_models[:4]: # loop through models to perform tagging
            model = TimeSeriesKMeans.from_pickle(os.path.join(path, f'{name}.p'))
            tagging[name] = model.predict(dataframe_to_ts(data=ts, series=name.replace('clust_','')))
            tagging[name] = tagging[name].astype(int)
        all_stocks_tagging.append(tagging)
        print(f'[{datetime.datetime.now()}] Completed {train_test} data tagging for stock {stock_id}')
    all_stocks_tagging = pd.concat(all_stocks_tagging, axis=0).reset_index(drop=True)
    return all_stocks_tagging

def dataframe_to_ts(data, series):
    ts = [data[(data.stock_id==stock_id) & (data.time_id==time_id) & (data[series].isnull()==False)][series].tolist() for stock_id, time_id in data[['stock_id','time_id']].drop_duplicates().to_records(index=False).tolist()]
    ts = to_time_series_dataset(ts)
    return ts

# create the new features
# tagging_train = gen_ts_clustering_feats(train_test='train', n_stock=N_STOCK)
if TRAIN==True:
    tagging_train = pd.read_csv('/kaggle/input/stock-cluster-tagging-train/tagging_train.csv') # pre-computed for training set
    train = pd.merge(train, tagging_train, how='left', on=['stock_id','time_id'])
    cols = [col for col in train.columns if col[:6]=='clust_']
    train[cols] = train[cols].fillna(0)

tagging_test = tagging_cluster_all_stocks(n_stock=N_STOCK, train_test='test', n_time_id_sample=-1, seconds_sample_interval=10, remove_first_n_seconds=10, delete_unused_cols=True)
test = pd.merge(test, tagging_test, how='left', on=['stock_id','time_id'])
cols = [col for col in test.columns if col[:6]=='clust_']
test[cols] = test[cols].fillna(0)

Processing /kaggle/input/tslearn052/tslearn-0.5.2-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl


/opt/conda/lib/python3.7/site-packages/tslearn/clustering/kmeans.py:17: UserWarning: Scikit-learn <0.24 will be deprecated in a future release of tslearn
  "Scikit-learn <0.24 will be deprecated in a "


[2021-09-27 05:17:34.791585] Completed test data tagging for stock 0
CPU times: user 2.58 s, sys: 698 ms, total: 3.27 s
Wall time: 32.4 s


In [8]:
'''
Add Stock Clustering as categorical feature
'''
from sklearn.cluster import AgglomerativeClustering
# prepare data for clustering
data = pd.pivot_table(train, values='target', index='time_id', columns='stock_id', aggfunc=np.sum)
data = data.fillna(data.mean()).T
# model fitting
model = AgglomerativeClustering(n_clusters=6, linkage='ward')
label = model.fit_predict(data)
print(pd.Series(label).value_counts())
# create stock-label mapping table
data['stock_clustering_label'] = label
data = data.reset_index().rename(columns={'index':'stock_id'})
data = data[['stock_id','stock_clustering_label']]
# add label to train / test
if TRAIN==True:
    train = pd.merge(train, data, how='left', on='stock_id')
test = pd.merge(test, data, how='left', on='stock_id')

3    33
2    30
5    27
1    13
0     8
4     1
dtype: int64


In [9]:
'''
Add Time Clustering as categorical feature
'''

from sklearn.cluster import KMeans
feature_list = ['wap1_sum','log_return1_realized_volatility','log_return1_realized_volatility_200','log_return1_realized_volatility_400','total_volume_sum','volume_imbalance_sum','trade_size_sum','trade_order_count_sum','trade_log_return_realized_volatility']
log_feats = ['log_return1_realized_volatility','log_return1_realized_volatility_200','log_return1_realized_volatility_400','total_volume_sum','trade_size_sum','trade_order_count_sum','trade_log_return_realized_volatility']

if TRAIN==True:
    # prepare data for clustering
    train_time_feature = train.groupby('time_id')[feature_list].mean().reset_index()
    train_time_feature[log_feats] = np.log(train_time_feature[log_feats])
    train_time_feature = fill_inf_nan(train_time_feature)
    # kmeans clustering
    kmeans = KMeans(n_clusters=8, random_state=SEED).fit(train_time_feature[feature_list])
    # transformation for train
    train_time_feature['time_clustering_label'] = kmeans.predict(train_time_feature[feature_list])
    train_time_feature = train_time_feature[['time_id','time_clustering_label']]
    train = pd.merge(train, train_time_feature, how='left', on='time_id')
    # transformation for test
    test_time_feature = test.groupby('time_id')[feature_list].mean().reset_index()
    test_time_feature[log_feats] = np.log(test_time_feature[log_feats])
    test_time_feature = fill_inf_nan(test_time_feature)
    test_time_feature['time_clustering_label'] = kmeans.predict(test_time_feature[feature_list])
    test_time_feature = test_time_feature[['time_id','time_clustering_label']]
    test = pd.merge(test, test_time_feature, how='left', on='time_id')
    # save model
    pickle.dump(kmeans, open(f'time_clustering_kmeans_model.p', 'wb'))
else:
    # transformation for test
    test_time_feature = test.groupby('time_id')[feature_list].mean().reset_index()
    test_time_feature[log_feats] = np.log(test_time_feature[log_feats])
    test_time_feature = fill_inf_nan(test_time_feature)
    kmeans = pickle.load(open(os.path.join(FE_PATH, f'time_clustering_kmeans_model.p'), 'rb'))
    test_time_feature['time_clustering_label'] = kmeans.predict(test_time_feature[feature_list])
    test_time_feature = test_time_feature[['time_id','time_clustering_label']]
    test = pd.merge(test, test_time_feature, how='left', on='time_id')

In [10]:
%%time
'''
Time-Series LSTM AutoEncoding features
'''
from tensorflow.keras import metrics
from tensorflow import keras
import tensorflow as tf
from numpy.random import seed
from tensorflow.keras.models import Model

# function to load a book/trade train/test single stock file
def load_single_stock(stock_id, train_test, book_trade):
    path = f'/kaggle/input/{data_path}/{book_trade}_{train_test}.parquet/stock_id={str(stock_id)}'
    filename = os.path.join(path, os.listdir(path)[0])
    return pd.read_parquet(filename)

def preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval):
    # load data
    book = load_single_stock(stock_id, train_test, 'book')
    trade = load_single_stock(stock_id, train_test, 'trade')
    # ffill and bfill
    time_id_list =  pd.DataFrame({'time_id':book.time_id.unique().tolist()})
    seconds_in_bucket_list = pd.DataFrame({'seconds_in_bucket':range(600)})
    base = time_id_list.merge(seconds_in_bucket_list, how='cross')
    book = base.merge(book, how='left', on=['time_id','seconds_in_bucket'])
    book = book.ffill().bfill()
    trade = base.merge(trade, how='left', on=['time_id','seconds_in_bucket'])
    trade[['price']] = trade[['price']].ffill().bfill()
    trade[['size','order_count']] = trade[['size','order_count']].fillna(0)
    # joining book and trade
    df = pd.merge(book, trade, how='inner', on=['time_id','seconds_in_bucket'])
    # sampling time_id
    if n_time_id_sample >= 0:
        time_id_samples = random.sample(df['time_id'].unique().tolist(), n_time_id_sample)
        df = df[df.time_id.isin(time_id_samples)]
    # smoothing all time series
    ts = [c for c in df if c not in ['time_id','seconds_in_bucket']]
    for c in ts:
        df[c] = df.groupby('time_id')[c].apply(lambda s: s.rolling(window=50).mean())
    # seconds interval filtering
    idx = df.index.tolist()[::seconds_sample_interval]
    df = df.iloc[idx,:]
    # remove NA entries due to moving average
    df = df[df.isnull().sum(axis=1)==0].reset_index(drop=True)
    # select only relevant columns
    df['stock_id'] = stock_id
    df = df[['stock_id'] + [c for c in df if c not in ['stock_id']]]
    df = df.drop('seconds_in_bucket', axis=1)
    return df

def preprocess_all_stocks(n_stock, train_test, n_time_id_sample, seconds_sample_interval, reduce_mem):
    stock_list = sorted([int(x.split('=')[1]) for x in os.listdir(f'/kaggle/input/{data_path}/book_{train_test}.parquet')])
    stock_list = stock_list[:n_stock]
    df = []
    for stock_id in stock_list:
        df.append(preprocess_single_stock(stock_id, train_test, n_time_id_sample, seconds_sample_interval))
        if stock_list.index(stock_id) % 10 == 0:
            print(f'[{datetime.datetime.now()}] Processed {stock_list.index(stock_id) % 10} stock_ids')
    df = pd.concat(df, axis=0).reset_index(drop=True)
    print(f'[{datetime.datetime.now()}] Completed data preprocessing')
    
    # scaling
    num_feats = [c for c in df if c not in ['stock_id','time_id']]
    scalers = pickle.load(open(os.path.join('/kaggle/input/volatility-time-series-autoencoder-data', 'ts_ae_stdscalers.p'), 'rb'))
    for c in num_feats:
        df[[c]] = scalers[c].transform(df[[c]])
        print(f'[{datetime.datetime.now()}] Scaled {c}')
    print(f'[{datetime.datetime.now()}] Completed data scaling')
    
    # memory reduction
    if reduce_mem==True:
        df = reduce_mem_usage(df)
        print(f'[{datetime.datetime.now()}] Completed memory reduction')
    return df

def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:5] == 'float':
                if c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    return df

# create dataset
data_test = preprocess_all_stocks(n_stock=N_STOCK, train_test='test', n_time_id_sample=-1, seconds_sample_interval=10, reduce_mem=True)
print(f'[{datetime.datetime.now()}] Completed data preprocessing')

# remove price
# data_train = data_train.drop(['ask_price1','bid_price2','ask_price2'], axis=1)
data_test = data_test.drop(['ask_price1','bid_price2','ask_price2'], axis=1)

# convert to np array
data_test_keys = data_test[['stock_id','time_id']].drop_duplicates()
n_split = data_test_keys.shape[0]
data_test = np.array(np.split(data_test.iloc[:,2:].values, n_split))
print(f'Shape of data_test is {data_test.shape}')

# load pre-trained encoder
path = os.path.join('/kaggle/input/volatility-ts-encoding-version-5', 'TimeSeries_Encoder')
ts_encoder = tf.keras.models.load_model(path)
encoding_dim = 32

# encoding
if TRAIN==True:
    data_train_keys = pd.read_csv('/kaggle/input/volatility-ts-encoding-version-5/train_encoded.csv')
    print(f'Shape of train encoding is {data_train_keys.shape}')
    train = pd.merge(train, data_train_keys, how='left', on=['stock_id','time_id'])
    del data_train_keys
    gc.collect()
data_test_keys[[f'ts_ae{i}' for i in range(encoding_dim)]] = ts_encoder.predict(data_test)
print(f'Shape of test encoding is {data_test_keys.shape}')
test = pd.merge(test, data_test_keys, how='left', on=['stock_id','time_id'])
del data_test, data_test_keys
gc.collect()

[2021-09-27 05:17:57.394309] Processed 0 stock_ids
[2021-09-27 05:17:57.395068] Completed data preprocessing
[2021-09-27 05:17:57.407340] Scaled bid_price1
[2021-09-27 05:17:57.410803] Scaled ask_price1
[2021-09-27 05:17:57.414088] Scaled bid_price2
[2021-09-27 05:17:57.417365] Scaled ask_price2
[2021-09-27 05:17:57.420620] Scaled bid_size1
[2021-09-27 05:17:57.423847] Scaled ask_size1
[2021-09-27 05:17:57.427062] Scaled bid_size2
[2021-09-27 05:17:57.430265] Scaled ask_size2
[2021-09-27 05:17:57.433454] Scaled price
[2021-09-27 05:17:57.436655] Scaled size
[2021-09-27 05:17:57.439891] Scaled order_count
[2021-09-27 05:17:57.439962] Completed data scaling
Memory usage after optimization is: 0.00 MB
Decreased by 41.4%
[2021-09-27 05:17:57.450609] Completed memory reduction
[2021-09-27 05:17:57.450716] Completed data preprocessing
Shape of data_test is (1, 55, 8)
Shape of train encoding is (428932, 34)
Shape of test encoding is (1, 34)
CPU times: user 13.8 s, sys: 1.13 s, total: 15 s
Wal

698

In [11]:
# Create "tau" features
if TRAIN==True:
    train['size_tau'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique'] )
    train['size_tau_400'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_400'] )
    train['size_tau_300'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_300'] )
    train['size_tau_200'] = np.sqrt( 1/ train['trade_seconds_in_bucket_count_unique_200'] )
    train['size_tau2'] = np.sqrt( 1/ train['trade_order_count_sum'] )
    train['size_tau2_400'] = np.sqrt( 0.33/ train['trade_order_count_sum'] )
    train['size_tau2_300'] = np.sqrt( 0.5/ train['trade_order_count_sum'] )
    train['size_tau2_200'] = np.sqrt( 0.66/ train['trade_order_count_sum'] )
    train['size_tau2_d'] = train['size_tau2_400'] - train['size_tau2']
    cols = [col for col in train.columns if 'tau' in col]
    train[cols] = train[cols].replace([np.inf, -np.inf], np.nan)
    train[cols] = train[cols].fillna(train[cols].mean())

test['size_tau'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique'] )
test['size_tau_400'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_400'] )
test['size_tau_300'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_300'] )
test['size_tau_200'] = np.sqrt( 1/ test['trade_seconds_in_bucket_count_unique_200'] )
test['size_tau2'] = np.sqrt( 1/ test['trade_order_count_sum'] )
test['size_tau2_400'] = np.sqrt( 0.33/ test['trade_order_count_sum'] )
test['size_tau2_300'] = np.sqrt( 0.5/ test['trade_order_count_sum'] )
test['size_tau2_200'] = np.sqrt( 0.66/ test['trade_order_count_sum'] )
test['size_tau2_d'] = test['size_tau2_400'] - test['size_tau2']
cols = [col for col in test.columns if 'tau' in col]
test[cols] = test[cols].replace([np.inf, -np.inf], np.nan)
test[cols] = test[cols].fillna(test[cols].mean())

In [12]:
'''
Stock cluster features
For each time_id, statistics for each of 5 clusters (for only that time_id) will be added
'''
from sklearn.cluster import KMeans
from collections import Counter

# clustering based on target correlation matrix
n_clust = 7
train_p = pd.read_csv(f'../input/{data_path}/train.csv')
train_p = train_p.pivot(index='time_id', columns='stock_id', values='target')
corr = train_p.corr()
kmeans = KMeans(n_clusters=n_clust, random_state=SEED).fit(corr.values)
print(Counter(kmeans.labels_))

# output the list of stocks for each cluster
stock_groups = []
ids = corr.index
for n in range(n_clust):
    stock_groups.append([(x-1) for x in ((ids+1)*(kmeans.labels_==n)) if x > 0])

def gen_clust_feats(stock_groups, train, test):
    df_train = []
    df_test = []
    for i in range(len(stock_groups)):
        group = stock_groups[i]
        newDf = train.loc[train['stock_id'].isin(group)]
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = f'c{i}'
        df_train.append(newDf)

        newDf = test.loc[test['stock_id'].isin(group)]    
        newDf = newDf.groupby(['time_id']).agg(np.nanmean)
        newDf.loc[:,'stock_id'] = f'c{i}'
        df_test.append(newDf)

    df_train = pd.concat(df_train).reset_index()
    df_train = df_train[[c for c in df_train if c!='target']]
    df_test = pd.concat(df_test).reset_index()

    df_test = pd.concat([df_test, df_train[(~df_train.stock_id.isin(df_test.stock_id)) & (df_train.time_id==5)]]) # suspicious; probably this is to ensure that test data contains all stocks; it works on the assumption that time_id 5 is not required for submission
    df_train = df_train.pivot(index='time_id', columns='stock_id')
    df_train.columns = ["_".join(x) for x in df_train.columns.ravel()]
    df_train.reset_index(inplace=True)

    df_test = df_test.pivot(index='time_id', columns='stock_id')
    df_test.columns = ["_".join(x) for x in df_test.columns.ravel()]
    df_test.reset_index(inplace=True)

    # add to base features
    target_cols = ['time_id'] + [f'{col}_c{i}' for i in [0,2,3,4,6] for col in ['log_return1_realized_volatility','total_volume_sum','trade_size_sum','trade_order_count_sum',
                                                                                'price_spread_sum','bid_spread_sum','ask_spread_sum','volume_imbalance_sum','bid_ask_spread_sum','size_tau2']]
    train = pd.merge(train, df_train[target_cols], how='left', on='time_id')
    test = pd.merge(test, df_test[target_cols], how='left', on='time_id')

    # free memory
    import gc
    del df_train, df_test
    gc.collect()
    return train, test

if TRAIN==True:
    train, test = gen_clust_feats(stock_groups=stock_groups, train=train[[c for c in train if c[-3:-1]!='_c']], test=test)
else:
    _ , test = gen_clust_feats(stock_groups=stock_groups, train=train, test=test)
    
train = train.fillna(train.mean()).fillna(0)
test = test.fillna(test.mean()).fillna(0)

Counter({3: 32, 0: 31, 4: 19, 2: 19, 6: 8, 1: 2, 5: 1})


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


In [13]:
# remove highly correlated features
if TRAIN==True:
    corr = train[[c for c in train if c[-3:-1]!='_c' and 
                  c[:6]!='clust_' and 
                  'clustering_label' not in c and 
                  c not in ['stock_id','time_id','row_id','target']]].corr()
    all_features = corr.columns.tolist()
    print(f'Original feats is {len(all_features)}')
    correlated_features = set()
    for i in range(len(corr.columns)):
        for j in range(i):
            if corr.iloc[i, j] >= 0.99:
                correlated_features.add(corr.columns[i])
    print(f'Removed feats is {len(correlated_features)}')
    train = train.drop(correlated_features, axis=1)
    pickle.dump(correlated_features, open(f'correlated_features.p', 'wb'))
else:
    correlated_features = pickle.load(open(os.path.join(FE_PATH, f'correlated_features.p'), 'rb'))
    
test = test.drop(correlated_features, axis=1)

Original feats is 245
Removed feats is 51


In [14]:
# final fillna
train = fill_inf_nan(train)
test = fill_inf_nan(test)

In [15]:
# Print number of features
colNames = [col for col in list(train.columns) if col not in ["stock_id", "time_id", "target", "row_id"]]
print(f'Number of features is {len(colNames)}')

Number of features is 250


### NN Dataset

In [16]:
# prepare NN dataset

# features that requires quantile transformation for NN model
nn_qt_feats = [col for col in train.columns if \
               col[-3:-1]!='_c' and \
               col[:6]!='clust_' and \
               'clustering_label' not in col and \
               col not in ['stock_id','time_id','row_id','target']]

if TRAIN==True:
    train_nn=train[nn_qt_feats].copy()
else:
    train_nn = pd.read_feather(os.path.join(FE_PATH, 'train_nn.f'))
test_nn=test[nn_qt_feats].copy()

# quantile transformation
if TRAIN==True:
    qt = QuantileTransformer(random_state=SEED, n_quantiles=2000, output_distribution='normal')
    qt.fit(train_nn[nn_qt_feats])
    train_nn[nn_qt_feats] = qt.transform(train_nn[nn_qt_feats])
    test_nn[nn_qt_feats] = qt.transform(test_nn[nn_qt_feats])
    pickle.dump(qt, open(f'quantile_transformer.p', 'wb'))
else:
    qt = pickle.load(open(os.path.join(FE_PATH, 'quantile_transformer.p'), 'rb'))
    test_nn[nn_qt_feats] = qt.transform(test_nn[nn_qt_feats])

# reset key columns
train_nn[['stock_id','time_id','row_id','target']] = train[['stock_id','time_id','row_id','target']].reset_index(drop=True)
test_nn[['stock_id','time_id','row_id']] = test[['stock_id','time_id','row_id']].reset_index(drop=True)

# add back stock and time clustering labels
stock_time_clust_label_feats = [c for c in train if 'clustering_label' in c]
train_nn[stock_time_clust_label_feats] = train[stock_time_clust_label_feats]
test_nn[stock_time_clust_label_feats] = test[stock_time_clust_label_feats]

# generate stock cluster features
if TRAIN==True:
    train_nn, test_nn = gen_clust_feats(stock_groups=stock_groups, train=train_nn, test=test_nn)
else:
    _ , test_nn = gen_clust_feats(stock_groups=stock_groups, train=train_nn, test=test_nn)


# fillna with mean
train_nn = fill_inf_nan(train_nn)
test_nn = fill_inf_nan(test_nn)

# stock_id label encoding (required for NN embedding)
nn_cat_cols = ['stock_id'] + stock_time_clust_label_feats
for col in nn_cat_cols:
    if TRAIN==True:
        encoder = LabelEncoder()
        encoder.fit(train_nn[col].values)
        train_nn[col] = encoder.transform(train_nn[col].values)
        test_nn[col] = encoder.transform(test_nn[col].values)
        pickle.dump(encoder, open(f'nn_label_encoder_{col}.p', 'wb'))
    else:
        encoder = pickle.load(open(os.path.join(FE_PATH, f'nn_label_encoder_{col}.p'), 'rb'))
        test_nn[col] = encoder.transform(test_nn[col].values)
        
# min max scaling
nn_num_feats = [c for c in train_nn if c not in ['stock_id','time_id','row_id','target'] and 'clustering_label' not in c]
if TRAIN==True:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaler.fit(train_nn[nn_num_feats])
    train_nn[nn_num_feats] = scaler.transform(train_nn[nn_num_feats])
    pickle.dump(scaler, open(f'nn_min_max_scaler.p', 'wb'))
    test_nn[nn_num_feats] = scaler.transform(test_nn[nn_num_feats])
else:
    scaler = pickle.load(open(os.path.join(FE_PATH, 'nn_min_max_scaler.p'), 'rb'))
    test_nn[nn_num_feats] = scaler.transform(test_nn[nn_num_feats])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:43: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:47: FutureWarning: Index.ravel returning ndarray is deprecated; in a future version this will return a view on self.


### TabNet Dataset

In [17]:
# define dataset for TabNet
train_tbn = train_nn.copy()
test_tbn = test_nn.copy()

# identify categorical and numerical columns
cat_cols = ['stock_id'] + stock_time_clust_label_feats
num_cols = [c for c in train_tbn if c not in ['stock_id','time_id','row_id','target'] and c not in stock_time_clust_label_feats]

# label encoding catergorical cols
for col in cat_cols:
    if TRAIN==True:
        encoder = LabelEncoder()
        encoder.fit(train_tbn[col].values)
        train_tbn[col] = encoder.transform(train_tbn[col].values)
        test_tbn[col] = encoder.transform(test_tbn[col].values)
        pickle.dump(encoder, open(f'tabnet_label_encoder_{col}.p', 'wb'))
    else:
        encoder = pickle.load(open(os.path.join(FE_PATH, f'tabnet_label_encoder_{col}.p'), 'rb'))
        test_tbn[col] = encoder.transform(test_tbn[col].values)
    
# standard scaling numerical cols
if TRAIN==True:
    scaler = StandardScaler()
    scaler.fit(train_tbn[num_cols])
    train_tbn[num_cols] = scaler.transform(train_tbn[num_cols])
    test_tbn[num_cols] = scaler.transform(test_tbn[num_cols])
    pickle.dump(scaler, open(f'tabnet_std_scaler.p', 'wb'))
else:
    scaler = pickle.load(open(os.path.join(FE_PATH, 'tabnet_std_scaler.p'), 'rb'))
    test_tbn[num_cols] = scaler.transform(test_tbn[num_cols])

In [18]:
# export
if TRAIN==True:
    train.reset_index(drop=True).to_feather('train.f')
    test.reset_index(drop=True).to_feather('test.f')
    train_nn.reset_index(drop=True).to_feather('train_nn.f')
    test_nn.reset_index(drop=True).to_feather('test_nn.f')
    train_tbn.reset_index(drop=True).to_feather('train_tbn.f')
    test_tbn.reset_index(drop=True).to_feather('test_tbn.f')

In [19]:
print(train.shape)
print(test.shape)
print(train_nn.shape)
print(test_nn.shape)
print(train_tbn.shape)
print(test_tbn.shape)

(428932, 254)
(3, 253)
(428932, 250)
(3, 249)
(428932, 250)
(3, 249)
